**Reference Link:** [RAG Systems Essentials (Analytics Vidhya)](https://courses.analyticsvidhya.com/courses/take/rag-systems-essentials/lessons/60148017-hands-on-deep-dive-into-rag-evaluation-metrics-generator-metrics-i)

## Custom LLM as a Judge (G-Eval)

G-Eval is a framework that uses LLMs with chain-of-thoughts (CoT) to evaluate LLM outputs based on __ANY__ custom criteria.

The G-Eval metric is the most versatile type of metric `deepeval` has to offer, and is capable of evaluating almost any use case with good accuracy.

Here you are free to describe your custom evaluation process in detail using prompts in `evaluation_steps`.

In `deepeval`, to use the GEval, you'll have to provide the following arguments when creating an `LLMTestCase`:

- `input` : Input Query (not used in the computation)
- `actual_output` : Actual LLM Response

You'll also need to supply any additional arguments such as `expected_output` and `context` if your evaluation criteria depends on these parameters.



![](https://i.imgur.com/IuODLKQ.png)





In [0]:
query = "What is AI?"
response = rag_chain_w_sources.invoke(query)
response

### Example:

In [0]:
retrieved_context = [doc.page_content for doc in response['context']]
retrieved_context

In [0]:
ai_response = response['response']
ai_response

In [0]:
ai_response = 'AI refers to machines mimicking human intelligence, such as problem-solving and learning, and includes applications like electric sheep and cyborg kittens'
ai_response

In [0]:
human_answer = """AI, also known as Artificial Intelligence is used to build complex systems for applications
                  like virtual assistants, robotics and autonomous vehicles."""
human_answer

In [0]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import GEval
from deepeval import evaluate
from deepeval.test_case import LLMTestCaseParams

test_case = LLMTestCase(
    input=response['question'],
    actual_output=ai_response,
    expected_output=human_answer,
    retrieval_context=retrieved_context
)

metric = GEval(
    threshold=0.5,
    model="gpt-4o",
    name="RAG Fact Checker",
    # NOTE: you can only provide either criteria or evaluation_steps, and not both
    evaluation_steps=[
        "Create a list of statements from 'actual output'",
        "Validate if they are relevant and answers the given question in 'input', penalize if any statements are irrelevant",
        "Also Validate if they exist in 'expected output', penalize if any statements are missing or factually wrong",
        "Also validate if these statements are grounded in the 'retrieval context' and penalize if they are missing or factually wrong",
        "Finally also penalize if any statements seem to be invented or made up and do not make sense factually given the 'input' and 'retrieval context'"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT,
                       LLMTestCaseParams.ACTUAL_OUTPUT,
                       LLMTestCaseParams.EXPECTED_OUTPUT,
                       LLMTestCaseParams.RETRIEVAL_CONTEXT],
    verbose_mode=True
)

result = evaluate([test_case], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)